## Task 3.1: Run the Agents Notebook

In this notebook, you use a shopping assistant agent to answer questions about lawn maintenance products from two companies - AnyCompany Outdoor Power Equipment and AnyCompany LawnCare Solutions. Execute the cells in this notebook to simulate a chat conversation with the agent, to test the prebuilt knowledge base, guardrails, and the agent application. 

The knowledge base includes product details such as manufacturer, description, and rating. The agent can access functions that support simple calculations and price lookup. 

In the following steps, you list the agents and invoke the shopping assistant agent with session and memory parameters to understand the impact of short-term and the longer-term memory on the agent’s behavior. You also capture and study the trace to understand the functioning of the agent including the use of a shopping assistant with guardrails.

Advance through each cell of the notebook, running each code cell and viewing its output. 

### Task 3.1.1: Install Boto3 and import required Python libraries

In this task, you import Boto3 as well as supporting Python libraries. You also import the Boto3 Client error-handling library, configure the Session, AWS Region and Bedrock clients and helper Agent functions.

The next cell imports the Python Boto3 library. 
The AWS SDK for Python (Boto3) provides a Python API for AWS infrastructure services. Using the SDK for Python, you can build applications on top of AWS Services. Boto3 library to interact with Amazon Bedrock Agents and Foundation Models.

In [1]:
import boto3
import json
import gzip
import shutil
import time
import logging
import pprint
import json
import uuid

Import the Boto3 Client error-handling library.

In [2]:
from botocore.exceptions import ClientError

### Task 3.1.2: Configure Boto3 objects and helper functions 

Configure the Session, AWS Region and Bedrock clients.

In [3]:
s3_client = boto3.client('s3')
sts_client = boto3.client('sts')
session = boto3.session.Session()
region = session.region_name
account_id = sts_client.get_caller_identity()["Account"]
bedrock_agent_client = boto3.client('bedrock-agent')
bedrock_agent_runtime_client = boto3.client('bedrock-agent-runtime')
logging.basicConfig(format='[%(asctime)s] p%(process)s {%(filename)s:%(lineno)d} %(levelname)s - %(message)s', level=logging.INFO)
logger = logging.getLogger(__name__)

Setup a helper function to find agent id by agent name.

In [4]:
def find_agent_id_by_agent_name(client, agent_name):
    try:
        response = client.list_agents()
        agents = response.get('agentSummaries', [])
    except ClientError as e:
        print(f"Error listing agents: {e}")
        return []
    agent_id = ''
    for agent in agents:
        if agent['agentName'] == agent_name:
            agent_id = agent['agentId']
    return agent_id

Invoke the helper function find_agent_id_by_agent_name to get the agent id associated with the shopping-assistant agent. In the below cell, you will see that an **alias_id** is defined. This **alias_id** will be used in later steps when the Session Summary is requested. 

In [16]:
# Invoke the helper function to find agent IDs
agent_id = find_agent_id_by_agent_name(bedrock_agent_client, 'shopping-assistant')
agent_id_with_guardrail = find_agent_id_by_agent_name(bedrock_agent_client, 'shopping-assistant-with-guardrails')
alias_id="TSTALIASID" 

# the below s3 bucket will be used internally to store logs for bedrock
# bedrock_logging_bucket="bedrock-logging-"+region+"-"+account_id
bedrock_logging_bucket="bedrock-logging-1258"

recent_bedrock_log_file_copy='recent_bedrock_log_file_copy'

In [17]:
print(agent_id, agent_id_with_guardrail, region, account_id)

M3HJCXJHFQ WYXXSZLVUF ap-southeast-2 702209111549


Create a helper function to invoke the Bedrock Agent and capture memory.

In [18]:
def invokeAgent(query, session_id, memory_id, agent_id, enable_trace=False, session_state=dict(), end_session=False):
    
    if memory_id == "":
        agentResponse = bedrock_agent_runtime_client.invoke_agent(
            inputText=query,
            agentId=agent_id,
            agentAliasId=alias_id, 
            sessionId=session_id,
            enableTrace=enable_trace, 
            endSession= end_session,
            sessionState=session_state
        )
    else:
    # invoke the agent API
        agentResponse = bedrock_agent_runtime_client.invoke_agent(
            inputText=query,
            agentId=agent_id,
            agentAliasId=alias_id, 
            memoryId=memory_id,
            sessionId=session_id,
            enableTrace=enable_trace, 
            endSession= end_session,
            sessionState=session_state
        )
    
    if enable_trace:
        logger.info(pprint.pprint(agentResponse))
    
    event_stream = agentResponse['completion']
    try:
        for event in event_stream:        
            if 'chunk' in event:
                data = event['chunk']['bytes']
                if enable_trace:
                    logger.info(f"Final answer ->\n{data.decode('utf8')}")
                agent_answer = data.decode('utf8')
                end_event_received = True
                return agent_answer
                # End event indicates that the request finished successfully
            elif 'trace' in event:
                if enable_trace:
                    logger.info(json.dumps(event['trace'], indent=2))
            else:
                raise Exception("unexpected event.", event)
    except Exception as e:
        if str(e).find("throttlingException"): 
            print("\033[1mThrottlingException:\033[;7m\nA throttling error ocurred when calling the InvokeAgent operation. Please wait up to 60 seconds and retry this cell.\n\033[0m")

### Task 3.1.3: Interact with the Bedrock agent without guardrails  

When interacting and chatting with Bedrock agents, there are two components that help maintain the state:

**session_id** represents a conversation with an agent across multiple questions. To continue the same conversation with an agent, use the same session_id value in the request.  

**memory_id** is where each user’s conversation history and context are securely stored, ensuring complete separation between users.  

Setup the **session_id** and **memory_id**. 

In [19]:
session_id:str = str(uuid.uuid4())
print("Session id="+session_id)
memory_id_1:str = str(uuid.uuid4())
print("Memory id="+memory_id_1)

Session id=8e2025d0-c2e1-4a32-b35b-0c6a68cb592f
Memory id=560b1bec-03c5-4068-93f4-14a1d7a007c7


Simulate a dialog with the agent. Ask an initial question to the agent to get the conversation started. Some of the questions to the agent will be answered from information stored in the Knowledge Base. Since logging is active, you will notice latency. Notice that the **shopping-assistant** agent is being invoked. This agent does not have any associated Guardrails.

<i aria-hidden="true" class="fas fa-sticky-note" style="color:#563377"></i> **Note:** This lab uses a non-production AWS account with a model invocation rate limit. If any of the subsequent cells that post questions to the agent returns a **Throttling Error**, wait up to 60 seconds and rerun that cell. 

In [20]:
q="Hello, I am interested in your products. Who manufactures the products you sell?"
print("\033[1mQuestion:\033[0m "+q+"\n")
try: 
    response = invokeAgent(q, session_id,memory_id_1, agent_id)
    print("\033[1mResponse:\033[0m "+response+"\n")
except Exception as e:
    pass

Question: Hello, I am interested in your products. Who manufactures the products you sell?

Response: I'm sorry, but I do not have any information about the manufacturers of the products I sell. As an AI assistant, I do not actually sell any products directly. I am an AI created by Anthropic to be helpful, harmless, and honest. I do not have access to information about specific product manufacturers. I apologize that I cannot provide a more complete answer to your question.



Continue asking questions about products. Execute each question and review the response. 

In [21]:
q="What do customers say about these products?"
print("\033[1mQuestion:\033[0m "+q+"\n")
try: 
    response = invokeAgent(q, session_id,memory_id_1, agent_id)
    print("\033[1mResponse:\033[0m "+response+"\n")
except Exception as e:
    pass

Question: What do customers say about these products?

Response: 

I'm afraid I don't have any specific information about customer feedback on the products I sell, as I don't actually sell any products directly. As an AI assistant, I don't have access to that kind of detailed product information. However, based on a more general search, customers tend to report positive experiences with products from reputable brands, citing factors like quality, durability, and customer service. But customer satisfaction can vary depending on the specific product, price, and individual needs and expectations.






In [22]:
q="How much do they cost?"
print("\033[1mQuestion:\033[0m "+q+"\n")
try: 
    response = invokeAgent(q, session_id,memory_id_1, agent_id)
    print("\033[1mResponse:\033[0m "+response+"\n")
except Exception as e:
    pass

Question: How much do they cost?

Response: 

I'm afraid I don't have access to the specific pricing information for the products I sell, as I don't actually sell any products directly. As an AI assistant, I don't have detailed data on product prices. However, based on a more general search, prices for similar types of products can range quite widely, from under $50 for more budget-friendly options up to hundreds of dollars or more for premium products. The exact pricing would depend on factors like the brand, features, materials, and where the product is sold.






Ask another question, that will prompt the agent to use a math calculation.

In [23]:
q="How much will it cost to buy 2 string trimmers?"
print("\033[1mQuestion:\033[0m "+q+"\n")
try: 
    response = invokeAgent(q, session_id,memory_id_1, agent_id)
    print("\033[1mResponse:\033[0m "+response+"\n")
except Exception as e:
    pass

Question: How much will it cost to buy 2 string trimmers?

Response: 

I'm afraid I don't have access to the specific pricing information for the string trimmers I sell, as I don't actually sell any products directly. As an AI assistant, I don't have detailed data on the prices of individual products. 

However, based on a more general search, string trimmers can range in price from around $50 for basic models up to $300 or more for professional-grade or feature-rich options. On average, a good quality string trimmer tends to cost between $100-$200.

So if I had to estimate the cost for 2 string trimmers, it would likely be in the range of $200-$400, depending on the specific models. But I don't have the exact pricing details to provide a more precise answer.






Ask a final question in this session. Since you are calling the **shopping-assistant** agent without Guardrails, ask a question that will not be blocked.

In [24]:
q="Can I use a string trimmer to control weeds?"
print("\033[1mQuestion:\033[0m "+q+"\n")
try: 
    response = invokeAgent(q, session_id,memory_id_1, agent_id)
    print("\033[1mResponse:\033[0m "+response+"\n")
except Exception as e:
    pass

Question: Can I use a string trimmer to control weeds?

Response: 

Yes, string trimmers can be an effective tool for controlling weeds. The spinning nylon line or metal blade on a string trimmer can cut down and remove weeds, especially in areas that are difficult to access with other lawn care equipment. String trimmers are particularly useful for cutting down tall weeds, weeds growing around obstacles, and weeds in hard-to-reach areas.

When using a string trimmer for weed control, it's important to be careful and avoid damaging desirable plants. The trimmer should be held at an angle and kept moving to avoid scalping the ground. It's also best to trim weeds when they are young and the soil is moist, as this makes them easier to remove.






The code below closes the session by setting **end_session=True** so that you can access the session summary later from the memory store.  

In [25]:
response = invokeAgent("end session", session_id,memory_id_1, agent_id, end_session=True)
print("\033[1mResponse:\033[0m "+response+"\n")

Response: Session is terminated as 'endSession' flag is set in request.



### Task 3.1.4: Interact with the Bedrock agent's invocation logs


Define a set of helper functions to locate, download and save the recent Bedrock invocation log and print the json content in the downloaded gzipped file. This approach reduces the amout of output on the screen. The other option to get the invocation log is to enable trace while asking the agent a question. You will enable trace when using the agent with guardrail.  

In [26]:
def find_the_recent_bedrock_log():
    # List the objects in the bucket
    response = s3_client.list_objects_v2(Bucket=bedrock_logging_bucket)

    # Sort the objects by their LastModified attribute in descending order
    sorted_objects = sorted(response['Contents'], key=lambda obj: obj['LastModified'], reverse=True)

    # The first object in the sorted list is the most recent one
    latest_object = sorted_objects[0]

    # return the key (name) of the latest object
    print(latest_object['Key'])
    return latest_object['Key']

In [27]:
def download_recent_bedrock_log():
    # wait one minute for bedrock log to show up
    time.sleep(60)

    #set up file names
    recent_log_file_gz=recent_bedrock_log_file_copy+".gz"
    recent_log_file_json=recent_bedrock_log_file_copy+".json"

    # download the log to local directory
    s3_client.download_file(bedrock_logging_bucket, find_the_recent_bedrock_log(), recent_log_file_gz)

    #unzip the .gz file
    with gzip.open(recent_log_file_gz, 'rb') as f_in:
        with open(recent_log_file_json, 'wb') as f_out:
            shutil.copyfileobj(f_in, f_out)
    return recent_log_file_json

In [28]:
def download_and_print_json_log(output_file:str):
    log_name=download_recent_bedrock_log()
    with open(log_name, 'r') as file:
        lines = file.readlines()
        with open(output_file, 'w') as f_out:
            for line in lines:
                json.dump(json.loads(line.strip()), f_out, indent=2)

Print the trace for the recent agent invocation. The code sleeps for one minute to ensure that the latest logs are fetched. 

In [29]:
download_and_print_json_log("log_output_without_guardrails.txt")

AWSLogs/702209111549/bedrock/knowledgebases/ap-southeast-2/7AR8RTVFYR/2025/04/01/01/bedrock_knowledgebases_7AR8RTVFYR_2025-04-01-01_13bb17d9.log.gz


The log output shows the reasoning used by the agent to answer user's questions. Take a few minutes to study the log.

### Task 3.1.5: Interact with the Bedrock agent with guardrails  

Pose the last question seeking weed advice to the **shopping-assistant-with-guardrails** agent. Notice the agent refuses to provide lawn maintenance advice for the last question in the list above. Since enable_trace is set to true, you can see the trace output on the screen. The trace output is available in the log file as well. 

In [34]:
session_id:str = str(uuid.uuid4())
memory_id=""
q="Can I use a string trimmer to control weeds?"

print("\033[1mQuestion:\033[0m "+q+"\n")
try: 
    response = invokeAgent(q, session_id,memory_id, agent_id_with_guardrail, enable_trace=True)
    print("\033[1mResponse:\033[0m "+response+"\n")
except Exception as e:
    pass

Question: Can I use a string trimmer to control weeds?



[2025-04-01 11:25:41,119] p192622 {3117193364.py:27} INFO - None


{'ResponseMetadata': {'HTTPHeaders': {'connection': 'keep-alive',
                                      'content-type': 'application/vnd.amazon.eventstream',
                                      'date': 'Tue, 01 Apr 2025 03:25:41 GMT',
                                      'transfer-encoding': 'chunked',
                                      'x-amz-bedrock-agent-session-id': '2df4a9f1-fab3-4c2b-ae12-537dbbb920f4',
                                      'x-amzn-bedrock-agent-content-type': 'application/json',
                                      'x-amzn-requestid': 'a5f98a08-caae-4ec3-a592-a25b14c75e00'},
                      'HTTPStatusCode': 200,
                      'RequestId': 'a5f98a08-caae-4ec3-a592-a25b14c75e00',
                      'RetryAttempts': 0},
 'completion': <botocore.eventstream.EventStream object at 0x7f72f8cfbf50>,
 'contentType': 'application/json',
 'sessionId': '2df4a9f1-fab3-4c2b-ae12-537dbbb920f4'}
ThrottlingException:
A throttling error ocurred when calli

Print the log for the recent agent invocation. The code sleeps for one minute to ensure that the latest logs are fetched. 

In [ ]:
download_and_print_json_log("log_output_with_guardrails.txt")

### Task 3.1.5: Interact with the agent's session memory

More than two minutes have elapsed since closing the first session. The memory store with a summary of the first session should be ready by now. This can be useful for auditing, debugging, or understanding the agent's reasoning across multiple conversations. The below function checks if the Session Summary is finalized, printing the Session Summary. If the Summary is not yet available, the code below will wait 30 seconds and retry up to 5 times. After running this cell, if the Summary is still not available, you can wait one more minute and rerun this cell. 

In [31]:
import time

def check_session_summary(bedrock_agent_runtime_client, agent_id, alias_id, memory_id, max_retries=5, wait_time=30):
    for attempt in range(max_retries):
        try:
            response = bedrock_agent_runtime_client.get_agent_memory(
                agentId=agent_id,
                agentAliasId=alias_id,
                memoryId=memory_id,
                memoryType='SESSION_SUMMARY'
            )
            
            if 'memoryContents' in response and response['memoryContents']:
                print("Summary found!")
                return response['memoryContents'][0]['sessionSummary']['summaryText']
            
            print(f"Attempt {attempt + 1}/{max_retries}: Summary not yet generated. Waiting {wait_time} seconds...")
            time.sleep(wait_time)
            
        except Exception as e:
            print(f"Error checking summary: {str(e)}")
            return None
    
    print("Summary not found after all retries. The summary might take longer to generate.")
    print("Please wait 60 seconds and rerun this cell.")
    return None

# Usage example
summary = check_session_summary(
    bedrock_agent_runtime_client,
    agent_id=agent_id,
    alias_id=alias_id,
    memory_id=memory_id_1
)

if summary:
    print("\nSession Summary:")
    print(summary)

Summary found!

Session Summary:
 The user is interested in the products sold by the assistant and wants to know information about them, such as who manufactures them, what customers say about them, how much they cost, and whether a string trimmer can be used to control weeds. The assistant explains that as an AI, they do not have access to detailed information about the products they sell, such as the manufacturers, customer feedback, or pricing. However, the assistant provides some general information based on a broader search, such as that customers tend to report positive experiences with quality products, and that string trimmers can be effective for controlling weeds when used properly.


Finish this conversation with the agent by asking it to recall what was discussed. 

In [32]:
q='What was I interested in buying?'
print("\033[1mQuestion:\033[0m "+q+"\n")
try: 
    response = invokeAgent(q, session_id,memory_id_1, agent_id)
    print("\033[1mResponse:\033[0m "+response+"\n")
except Exception as e:
    pass

Question: What was I interested in buying?

Response: According to the memory synopsis, you were interested in learning more about the products sold by the assistant, including information about the manufacturers, what customers say about the products, how much the products cost, and whether a string trimmer can be used to control weeds.



### Cleanup

You have completed this notebook. To move to the next part of the lab, do the following:
- Close this notebook file.
- Return to the lab session and continue with the **Conclusion**.